In [33]:
import pandas as pd 
import numpy as np 

In [8]:
customers = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Transactions.csv")
merged_df = pd.merge(pd.merge(transactions, customers, on='CustomerID', how='left'), products, on='ProductID', how='left')
merged_df

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,T00496,C0118,P037,2024-10-24 08:30:27,1,459.86,459.86,Jacob Holt,South America,2022-01-22,SoundWave Smartwatch,Electronics,459.86
996,T00759,C0059,P037,2024-06-04 02:15:24,3,1379.58,459.86,Mrs. Kimberly Wright,North America,2024-04-07,SoundWave Smartwatch,Electronics,459.86
997,T00922,C0018,P037,2024-04-05 13:05:32,4,1839.44,459.86,Tyler Haynes,North America,2024-09-21,SoundWave Smartwatch,Electronics,459.86
998,T00959,C0115,P037,2024-09-29 10:16:02,2,919.72,459.86,Joshua Hamilton,Asia,2024-11-11,SoundWave Smartwatch,Electronics,459.86


In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
customers = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Customers.csv")
products = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Products.csv")
transactions = pd.read_csv(r"C:\Users\MEGHANA\Downloads\Transactions.csv")

# Step 1: Merge the datasets
merged_df = pd.merge(pd.merge(transactions, customers, on='CustomerID', how='left'), products, on='ProductID', how='left')

# Step 2: Feature Engineering
# Aggregate transaction data for each customer
customer_transactions = merged_df.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': 'nunique'
}).reset_index()

# Merge with customer profile data (Region, etc.)
customer_profile = customers[['CustomerID', 'Region']]  # You can add more columns as needed
customer_data = pd.merge(customer_transactions, customer_profile, on='CustomerID', how='left')

# Step 3: Preprocess Data (Normalize numerical features)
scaler = StandardScaler()
numerical_features = ['TotalValue', 'Quantity', 'ProductID']  # You can add more features if necessary
customer_data[numerical_features] = scaler.fit_transform(customer_data[numerical_features])

# Convert categorical features (e.g., Region) into dummy variables
customer_data = pd.get_dummies(customer_data, columns=['Region'], drop_first=True)

# Step 4: Similarity Calculation (Cosine Similarity)
customer_features = customer_data.drop('CustomerID', axis=1)  # Dropping CustomerID as it's not a feature
cosine_sim = cosine_similarity(customer_features)

# Step 5: Recommendation System - Create a lookalike mapping
def recommend_similar_customers(input_customer_id, top_n=3):
    input_index = customer_data[customer_data['CustomerID'] == input_customer_id].index[0]
    similarity_scores = cosine_sim[input_index]
    
    # Sort the similarity scores in descending order, excluding the input customer
    similar_customer_indices = similarity_scores.argsort()[-top_n-1:-1][::-1]
    similar_customers = customer_data.iloc[similar_customer_indices]
    
    similar_customers['SimilarityScore'] = similarity_scores[similar_customer_indices]
    return similar_customers[['CustomerID', 'SimilarityScore']]

# Step 6: Generate Lookalike Map for customers C0001 to C0020
lookalike_map = {}
for customer_id in [f'C{i:04}' for i in range(1, 21)]:
    lookalike_customers = recommend_similar_customers(customer_id)
    lookalike_map[customer_id] = lookalike_customers.values.tolist()

# Step 7: Save the result into a CSV file
lookalike_df = pd.DataFrame([
    {'cust_id': key, 'lookalikes': value} 
    for key, value in lookalike_map.items()
])

lookalike_df.to_csv('Lookalike.csv', index=False)
print("Lookalike.csv has been saved.")


Lookalike.csv has been saved.


C:\Users\MEGHANA\AppData\Local\Temp\ipykernel_15148\2272029989.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_customers['SimilarityScore'] = similarity_scores[similar_customer_indices]
C:\Users\MEGHANA\AppData\Local\Temp\ipykernel_15148\2272029989.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_customers['SimilarityScore'] = similarity_scores[similar_customer_indices]
C:\Users\MEGHANA\AppData\Local\Temp\ipykernel_15148\2272029989.py:46: SettingWithCopyWarning: 
A value is trying t

In [24]:
lookalike_df = pd.read_csv('Lookalike.csv')
print(lookalike_df)

   cust_id                                         lookalikes
0    C0001  [['C0107', 0.9893878848465956], ['C0137', 0.98...
1    C0002  [['C0142', 0.9881467419364092], ['C0043', 0.97...
2    C0003  [['C0133', 0.9226192335483311], ['C0031', 0.86...
3    C0004  [['C0113', 0.9882333885531043], ['C0102', 0.97...
4    C0005  [['C0123', 0.9997811226617136], ['C0097', 0.99...
5    C0006  [['C0168', 0.9570729304999362], ['C0158', 0.90...
6    C0007  [['C0140', 0.9984538458917513], ['C0080', 0.99...
7    C0008  [['C0090', 0.9544410773515254], ['C0084', 0.94...
8    C0009  [['C0198', 0.9814560986255916], ['C0061', 0.96...
9    C0010  [['C0166', 0.9668651251682921], ['C0199', 0.94...
10   C0011  [['C0107', 0.9853983059570073], ['C0048', 0.98...
11   C0012  [['C0102', 0.9836972844533496], ['C0153', 0.97...
12   C0013  [['C0155', 0.9915035127378732], ['C0099', 0.98...
13   C0014  [['C0060', 0.9995404219771602], ['C0198', 0.99...
14   C0015  [['C0058', 0.996463297835375], ['C0033', 0.995...
15   C00

In [37]:
import pandas as pd
import ast

# Step 1: Read the Lookalike CSV file
lookalike_df = pd.read_csv('Lookalike.csv')

# Step 2: Filter the rows for the first 20 customers (C0001 to C0020)
lookalike_df_filtered = lookalike_df[lookalike_df['cust_id'].isin([f'C{i:04d}' for i in range(1, 21)])]

# Step 3: Extract the top 3 lookalikes for each customer and their similarity scores
top_lookalikes = {}

for _, row in lookalike_df_filtered.iterrows():
    cust_id = row['cust_id']
    lookalikes_list = ast.literal_eval(row['lookalikes'])  # Convert string list to actual list
    top_lookalikes[cust_id] = lookalikes_list[:3]  # Get top 3 lookalikes

# Step 4: Display the results
for cust_id, lookalikes in top_lookalikes.items():
    print(f"Customer {cust_id}:")
    for lookalike in lookalikes:
        print(f"  Lookalike: {lookalike[0]}, Similarity Score: {lookalike[1]}")
    print()


Customer C0001:
  Lookalike: C0107, Similarity Score: 0.9893878848465956
  Lookalike: C0137, Similarity Score: 0.9878583409566715
  Lookalike: C0191, Similarity Score: 0.9719460662437199

Customer C0002:
  Lookalike: C0142, Similarity Score: 0.9881467419364092
  Lookalike: C0043, Similarity Score: 0.977393593616262
  Lookalike: C0186, Similarity Score: 0.9495190142543483

Customer C0003:
  Lookalike: C0133, Similarity Score: 0.9226192335483311
  Lookalike: C0031, Similarity Score: 0.8608508774911415
  Lookalike: C0052, Similarity Score: 0.8592648813944818

Customer C0004:
  Lookalike: C0113, Similarity Score: 0.9882333885531043
  Lookalike: C0102, Similarity Score: 0.9713345085280436
  Lookalike: C0104, Similarity Score: 0.9711615751582937

Customer C0005:
  Lookalike: C0123, Similarity Score: 0.9997811226617136
  Lookalike: C0097, Similarity Score: 0.998355356744969
  Lookalike: C0078, Similarity Score: 0.9980977596475319

Customer C0006:
  Lookalike: C0168, Similarity Score: 0.957072

In [42]:
import pandas as pd

# Read the Lookalike CSV file
lookalike_df = pd.read_csv('Lookalike.csv')

# Define a threshold similarity score (e.g., 0.90)
threshold = 0.90

# Initialize variables to calculate accuracy
total_customers = len(lookalike_df)
correct_predictions = 0

# Step through each customer and evaluate their lookalikes
for _, row in lookalike_df.iterrows():
    lookalikes_list = ast.literal_eval(row['lookalikes'])
    
    # Count how many of the top 3 lookalikes have a similarity score above the threshold
    correct_for_customer = sum([score > threshold for _, score in lookalikes_list[:3]])
    
    # If at least one lookalike meets the threshold, consider the prediction correct
    if correct_for_customer > 0:
        correct_predictions += 1

# Calculate the accuracy
accuracy = correct_predictions / total_customers
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 95.00%
